# Model building: predicting neighbourhood rating and community belonging

In [1]:
# Libraries
import pandas as pd
import numpy as np
from janitor import clean_names
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Data preparation

The approach will be different from data preparation in the exploratory analysis. The aim will be to preserve as many data points as possible, whist reducing the cardinality of the many categorical variables. Given the targets to be predicted, only two datasets will be necessary, as they both contain the same columns except for the two dependent variables, neighbourhood rating and community belonging.

In [2]:
# Read in neighbourhood ratings
neighbourhood = pd.read_csv("data/neighbourhood_rating.csv").clean_names()

# Read in community ratings
community = pd.read_csv("data/community_belonging.csv").clean_names()

In [3]:
# Joining the two datasets.
survey = (
    pd.merge(neighbourhood, community, how='outer')
    .query("featurecode != 'S92000003' & measurement == 'Percent'")
    .drop(columns=['measurement', 'units', 'featurecode'])
    .fillna('All')
)

survey.sample(10)

,datecode,value,neighbourhood_rating,gender,urban_rural_classification,simd_quintiles,type_of_tenure,household_type,ethnicity,walking_distance_to_nearest_greenspace,community_belonging
11087,2014,0.0,No opinion,All,All,All,All,All,White,All,All
72160,2019,38.0,All,All,All,All,Owned Mortgage/Loan,All,All,All,Very strongly
24268,2018,7.0,Fairly poor,All,Urban,All,All,All,All,All,All
762,2015,2.0,Fairly poor,All,All,All,All,All,All,Less than 10 minutes,All
71342,2018,39.0,All,All,All,All,Owned Mortgage/Loan,All,All,All,Very strongly
23799,2016,1.0,Fairly poor,All,Urban,All,All,All,All,All,Don't know
3735,2019,49.0,Very good,All,All,All,All,All,All,Less than 10 minutes,All
48849,2014,47.0,All,All,Urban,All,All,All,All,All,Fairly strongly
60214,2018,29.0,All,All,All,All,All,With Children,All,All,Very strongly
45764,2019,3.0,All,Male,All,All,All,All,All,All,Not at all strongly


As already mentioned, many of the variables are categorical and show high cardinality, but do not fully explain the variation in the percentage of adults that fall into that category (see exploratory analysis report for more details). This is particularly true of our targets: community belonging and neighbourhood rating. For these reasons, many features will be engineered into binary classifiers that will hopefully better capture data variation, and simplify model building.

Additionally, confidence intervals and region codes have been dropped and the data points taken as country-wide statistical variation. In order to drop these two columns upper and lower bounds for the confidence intervals and the feature code for Scotland as a whole have been filtered out.

## Feature engineering

The following features will be re-binned or engineered:

- `walking_distance_to_nearest_greenspace` will become `green_access`, 1 for those who live within 10 min of a green space, 0 otherwise.

- `neighbourghood_rating` and `community_belonging` will become `good_neighbourhood` and `good_community`, 1 if they are very/fairly good/strong, and 0 otherwise.

- `type_of_tenure` will be replaced by `owner` and `tenant` and their respective binary encoding.

- `household_type` will be replaced by `pensioners` and `children` and their respective binary encoding.

- The rest of the variables will also be binary encoded.

In [4]:
# Good neighbourhood rating
survey.loc[:, 'good_neighbourhood'] = np.where(
    survey['neighbourhood_rating'].isin(['Very good', 'Fairly good']),
    1, 0)

# Good community belonging feeling
survey.loc[:, 'good_community'] = np.where(
    survey['community_belonging'].isin(['Very strongly', 'Fairly strongly']),
    1, 0)

# Green space access
survey.loc[:, 'green_access'] = np.where(
    survey['walking_distance_to_nearest_greenspace'] == 'Less than 10 minutes',
    1, 0)

# Owners
survey.loc[:, 'owners'] = np.where(
    survey['type_of_tenure'].isin(['Owned Outright', 'Owned Mortgage/Loan']),
    1, 0)

# Tenants
survey.loc[:, 'tenants'] = np.where(
    survey['type_of_tenure'].isin(['Social Rented', 'Private Rented']),
    1, 0)

# Pensioners
survey.loc[:, 'pensioners'] = np.where(
    survey['household_type'] == 'Pensioners',
    1, 0)

# children
survey.loc[:, 'children'] = np.where(
    survey['household_type'] == 'With Children',
    1, 0)

# Urban
survey.loc[:, 'urban'] = np.where(
    survey['urban_rural_classification'] == 'Urban',
    1, 0)

# Female
survey.loc[:, 'female'] = np.where(
    survey['gender'] == 'Female',
    1, 0)

# Lowest quintile
survey.loc[:, 'lowest_quintile'] = np.where(
    survey['simd_quintiles'] == '20% most deprived',
    1, 0)

# White ethnicity
survey.loc[:, 'white_ethnicity'] = np.where(
    survey['ethnicity'] == 'White',
    1, 0)

survey = survey.drop(columns=[
    'neighbourhood_rating', 'gender', 'urban_rural_classification',
    'simd_quintiles', 'type_of_tenure', 'household_type', 'ethnicity',
    'walking_distance_to_nearest_greenspace', 'community_belonging']).copy()

In [5]:
survey.sample(20)

,datecode,value,good_neighbourhood,good_community,green_access,owners,tenants,pensioners,children,urban,female,lowest_quintile,white_ethnicity
29752,2018,85.0,1,0,0,0,0,1,0,0,0,0,0
55247,2018,36.0,0,1,0,0,0,0,0,0,0,0,1
60907,2014,2.0,0,0,0,0,0,0,0,0,0,0,1
71730,2013,40.0,0,1,0,1,0,0,0,0,0,0,0
28186,2019,54.0,1,0,0,0,0,0,0,0,0,0,0
40283,2015,3.0,0,0,1,0,0,0,0,0,0,0,0
30517,2019,4.0,0,0,0,0,0,0,0,1,0,0,0
17956,2017,1.0,0,0,0,0,0,0,0,1,0,0,0
49142,2013,28.0,0,1,0,0,0,0,0,0,0,1,0
77368,2018,1.0,0,0,0,0,1,0,0,0,0,0,0


In [6]:
survey.good_neighbourhood.value_counts()[1] / survey.good_neighbourhood.value_counts()[0]

0.3353490929081913

By encoding data in this way, we can treat the constant value 'All' as the absence of a feature rather than a feature being held constant. With enough data points this will be sufficient to generate predictions, albeit the model may not be useful to explain why certain features correlate with higher ratings. Note however that this feature engineering introduces an imbalance, as the split between 1s and 0s is not close to 50%.

# Splitting test and train sets

Given that this data is dated, let us split it by date instead of at random. 2019 will become the test set and the model will be trained with the rest.

In [7]:
survey.datecode.value_counts()

2013    3560
2014    3540
2019    3483
2017    3482
2015    3462
2016    3442
2018    3321
Name: datecode, dtype: int64

Each year corresponds approximately with 1/7 th of the data, or 14%. For the train and test sets, let us drop both ratings from the train set as there might be a strong correlation between how people rate their community and their neighbourhood, so using each rating to predict the other would be circular.

In [8]:
# Train and test split
train = survey.query("datecode != 2019").drop('datecode', axis=1)

test = survey.query("datecode == 2019").drop('datecode', axis=1)

# Now drop both targets from the predictor sets
X_test = test.drop(columns=['good_neighbourhood', 'good_community'])
X_train = train.drop(columns=['good_neighbourhood', 'good_community'])

# Create both targets
community_test = test.good_community
community_train = train.good_community

neighbourhood_test = test.good_neighbourhood
neighbourhood_train = train.good_neighbourhood

# Building the pipeline for the logistic regression

In [9]:
# The ColumnTransformer allows for treating incoming data differently depending on its type (categorical, or numerical in the case of the value column)
log_pipe = Pipeline([
    ('split', ColumnTransformer([
        ('green_access', 
         OneHotEncoder(categories=[survey.green_access.unique()], drop='first', handle_unknown='error'), ['green_access']),
        ('owners', 
         OneHotEncoder(categories=[survey.owners.unique()], drop='first', handle_unknown='error'), ['owners']), 
        ('tenants', 
         OneHotEncoder(categories=[survey.tenants.unique()], drop='first', handle_unknown='error'), ['tenants']), 
        ('pensioners', 
         OneHotEncoder(categories=[survey.pensioners.unique()], drop='first', handle_unknown='error'), ['pensioners']),
         ('children', 
         OneHotEncoder(categories=[survey.children.unique()], drop='first', handle_unknown='error'), ['children']),
        ('urban', 
         OneHotEncoder(categories=[survey.urban.unique()], drop='first', handle_unknown='error'), ['urban']),
         ('female', 
         OneHotEncoder(categories=[survey.female.unique()], drop='first', handle_unknown='error'), ['female']),
         ('lowest_quintile', 
         OneHotEncoder(categories=[survey.lowest_quintile.unique()], drop='first', handle_unknown='error'), ['lowest_quintile']),
         ('white_ethnicity', 
         OneHotEncoder(categories=[survey.white_ethnicity.unique()], drop='first', handle_unknown='error'), ['white_ethnicity']),
        ('value', Pipeline([
                ('scale', StandardScaler())
            ]), ['value'])
    ])),
    # ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
    ('classifier', LogisticRegression())
])

# Fitting and scoring the models

## Neighbourhood ratings model

In [10]:
# Fitting the model to the train set for the neighbourhood rating
log_pipe.fit(X_train, neighbourhood_train);

In [11]:
print('MAE train for Logistic Regression:', mean_absolute_error(neighbourhood_train, log_pipe.predict(X_train)))
print('MAE test for Logistic Regression:', mean_absolute_error(neighbourhood_test, log_pipe.predict(X_test)))

MAE train for Logistic Regression: 0.18277502763493056
MAE test for Logistic Regression: 0.18863049095607234


The mean absolute error for the test and train sets is very similar, suggesting that the model is not over-fitting.

In [12]:
print(f"Logistic Regression model train mean accuracy: {cross_val_score(log_pipe, X_train, community_train, scoring='accuracy', cv=5).mean()}.");
print(f"Logistic Regression model test mean accuracy: {cross_val_score(log_pipe, X_test, community_test, scoring='accuracy', cv=5).mean()}.");

Logistic Regression model train mean accuracy: 0.6525693203208068.
Logistic Regression model test mean accuracy: 0.6609582941671202.


Accuracy is quite good for this model, and similar in the test and train sets.

In [13]:
print('AUC train for Logistic Regression:', roc_auc_score(neighbourhood_train, log_pipe.predict(X_train)))
print('AUC test for Logistic Regression:', roc_auc_score(neighbourhood_test, log_pipe.predict(X_test)))

AUC train for Logistic Regression: 0.7191536335071496
AUC test for Logistic Regression: 0.7083255198889017


The area under the ROC curve metric shows the relationship between the false positive rate and the true positive rate. A value of around 0.5 would signify that the model is as good at predicting positives as a coin flip.

## Community belonging model

In [14]:
# Fitting the model to the train set for the community belonging rating
log_pipe.fit(X_train, community_train);

In [15]:
print('MAE train for Logistic Regression:', mean_absolute_error(community_train, log_pipe.predict(X_train)))
print('MAE test for Logistic Regression:', mean_absolute_error(community_test, log_pipe.predict(X_test)))

MAE train for Logistic Regression: 0.3210458018935935
MAE test for Logistic Regression: 0.3181165661785817


In [16]:
print(f"Logistic Regression model train mean accuracy: {cross_val_score(log_pipe, X_train, community_train, scoring='accuracy', cv=5).mean()}.")
print(f"Logistic Regression model test mean accuracy: {cross_val_score(log_pipe, X_test, community_test, scoring='accuracy', cv=5).mean()}.")

Logistic Regression model train mean accuracy: 0.6525693203208068.
Logistic Regression model test mean accuracy: 0.6609582941671202.


In [17]:
print('AUC train for Logistic Regression:', roc_auc_score(community_train, log_pipe.predict(X_train)))
print('AUC test for Logistic Regression:', roc_auc_score(community_test, log_pipe.predict(X_test)))

AUC train for Logistic Regression: 0.4771463868970735
AUC test for Logistic Regression: 0.47564955902309647


For community belonging, the model does not perform well. Accuracy is low and the AUC value shows that the true / false positive rate is as good as a coin flip.

# Second approach: random forest classifier.

Given that the feature engineering and logistic regression has not been successful in the goal of creating an accurate model with regards to community belonging, another approach will be considered.

In [18]:
raw_survey = (
    pd.merge(neighbourhood, community, how='outer')
    .query("featurecode != 'S92000003' & measurement == 'Percent'")
    .drop(columns=['measurement', 'units', 'featurecode'])
    .fillna('All')
)

raw_survey.sample(10)

,datecode,value,neighbourhood_rating,gender,urban_rural_classification,simd_quintiles,type_of_tenure,household_type,ethnicity,walking_distance_to_nearest_greenspace,community_belonging
30999,2018,1.0,Very poor,All,All,All,Private Rented,All,All,All,All
26800,2013,10.0,Fairly poor,All,All,All,All,With Children,All,All,All
3144,2014,28.0,Fairly good,All,All,All,All,All,All,Less than 10 minutes,All
54076,2019,19.0,All,Male,All,All,All,All,All,All,Not very strongly
4646,2019,0.0,No opinion,Male,All,All,All,All,All,All,All
27912,2016,2.0,Fairly poor,All,All,All,All,Pensioners,All,All,Don't know
1062,2019,1.0,No opinion,All,All,All,All,All,All,Less than 10 minutes,All
21607,2018,55.0,Very good,All,All,All,All,Adults,All,All,All
20910,2017,4.0,Very poor,All,All,All,All,Adults,All,All,Not at all strongly
46255,2013,43.0,All,Female,All,All,All,All,All,All,Fairly strongly


In [19]:
# ProfileReport(raw_survey)

In [20]:
rf_pipe = Pipeline([
    ('split', ColumnTransformer([
        ('gender', 
         OneHotEncoder(categories=[raw_survey.gender.unique()], drop='first', handle_unknown='error'), ['gender']), 
        ('urban_rural_classification', 
         OneHotEncoder(categories=[raw_survey.urban_rural_classification.unique()], drop='first', handle_unknown='error'), ['urban_rural_classification']), 
        ('simd_quintiles', 
         OneHotEncoder(categories=[raw_survey.simd_quintiles.unique()], drop='first', handle_unknown='error'), ['simd_quintiles']),
         ('type_of_tenure', 
         OneHotEncoder(categories=[raw_survey.type_of_tenure.unique()], drop='first', handle_unknown='error'), ['type_of_tenure']),
        ('household_type', 
         OneHotEncoder(categories=[raw_survey.household_type.unique()], drop='first', handle_unknown='error'), ['household_type']),
         ('ethnicity', 
         OneHotEncoder(categories=[raw_survey.ethnicity.unique()], drop='first', handle_unknown='error'), ['ethnicity']),
         ('walking_distance_to_nearest_greenspace', 
         OneHotEncoder(categories=[raw_survey.walking_distance_to_nearest_greenspace.unique()], drop='first', handle_unknown='error'), ['walking_distance_to_nearest_greenspace']),
        ('value', Pipeline([
                ('scale', StandardScaler())
            ]), ['value'])
    ])),
    ('classifier', RandomForestClassifier())
])

In [21]:
# Train and test split
train = raw_survey.query("datecode != 2019").drop('datecode', axis=1)

test = raw_survey.query("datecode == 2019").drop('datecode', axis=1)

# Now drop both targets from the predictor sets
X_test = test.drop(columns=['neighbourhood_rating', 'community_belonging'])
X_train = train.drop(columns=['neighbourhood_rating', 'community_belonging'])

# Create both targets
community_test = test.community_belonging
community_train = train.community_belonging

neighbourhood_test = test.neighbourhood_rating
neighbourhood_train = train.neighbourhood_rating

# Start the label encoder to turn response types into numbers
le = LabelEncoder()

## Neighbourhood rating

In [22]:
# Fitting the model to the train set for the neighbourhood rating, including fitting to the encoded labels
le.fit(neighbourhood_train)
rf_pipe.fit(X_train, le.transform(neighbourhood_train));

In [23]:
# This is to show what the encoded labels are
le.classes_

array(['All', 'Fairly good', 'Fairly poor', 'No opinion', 'Very good',
       'Very poor'], dtype=object)

In [24]:
dummy_train = pd.get_dummies(X_train)

rf = RandomForestClassifier()
rf.fit(dummy_train, le.transform(neighbourhood_train))

RandomForestClassifier()

In [25]:
dummy_test = pd.get_dummies(X_test)
rf.score(dummy_test, le.transform(neighbourhood_test))

0.6118288831467126

Accuracy for the simple model is not high. Cross-validation accuracy is lower:

In [26]:
print(f"Logistic Regression model train mean accuracy: {cross_val_score(rf_pipe, X_train, le.transform(neighbourhood_train), scoring='accuracy', cv=5).mean()}.");
print(f"Logistic Regression model test mean accuracy: {cross_val_score(rf_pipe, X_test, le.transform(neighbourhood_test), scoring='accuracy', cv=5).mean()}.");

Logistic Regression model train mean accuracy: 0.4608964087362561.
Logistic Regression model test mean accuracy: 0.40677286894572795.


## Community rating

In [27]:
# Fitting the model to the train set for the neighbourhood rating, including fitting to the encoded labels
le.fit(community_train)
rf_pipe.fit(X_train, le.transform(community_train));

In [28]:
# This is to show what the encoded labels are
le.classes_

array(['All', "Don't know", 'Fairly strongly', 'Not at all strongly',
       'Not very strongly', 'Very strongly'], dtype=object)

In [29]:
dummy_train = pd.get_dummies(X_train)

rf = RandomForestClassifier()
rf.fit(dummy_train, le.transform(community_train))

RandomForestClassifier()

In [30]:
dummy_test = pd.get_dummies(X_test)
rf.score(dummy_test, le.transform(community_test))

0.5446454206144129

Accuracy for the simple model is not high. Same goes for the cross-validation and using the pipeline:

In [31]:
print(f"Logistic Regression model train mean accuracy: {cross_val_score(rf_pipe, X_train, le.transform(community_train), scoring='accuracy', cv=5).mean()}.");
print(f"Logistic Regression model test mean accuracy: {cross_val_score(rf_pipe, X_test, le.transform(community_test), scoring='accuracy', cv=5).mean()}.");

Logistic Regression model train mean accuracy: 0.3725703342899057.
Logistic Regression model test mean accuracy: 0.31986469104041954.


Overall, this would suggest that the random forest model is not very good either to predict community belonging.

# Summary

The goal is to predict community belonging and neighbourhood ratings in base of the demographic variables. Two approaches have been attempted.

The first one consisted in heavy feature engineering to encode all variables as binaries. Then, split the data by date, the train set being 2013-2018 and the test set 2019. Finally, fitting a logistic regression model to the train set. This model performed quite well to predict 2019 neighbourhood ratings and was subpar at predicting community belonging.

In the second approach features were not engineered. Data was split similarly by date, the train set then fitted to a random forest model. Both models performed worse for their target variables compared to their respective logistic models.